# Hurricans and Typhoons Classification.

## Objectives:
1. Read CSV file
2. Preprocess data
3. Prediction of Hurricanes or Typhoons using Decision Tree and Random Forrest

## Import Librairies

We import libraries as follows:

1. numpy for linear algebra
2. pandas for data processing
3. matplotlib, seaborn, shapely, geopandas for visualizarion
4. os for reading/writing data
5. re for regular expressions

In [1]:
# Ignore Warnings.
import warnings
warnings.filterwarnings("ignore")

# Import the libraries.

# Import pandas.
import pandas as pd
# Import numpy.
import numpy as np
# Import matplotlib.
import matplotlib.pyplot as plt
# Import seaborn.
import seaborn as sns
# Import regular expression.
import re
# import datetime.
import datetime as dt
# Import the data.

## Reading the CSV file
We are reading the file we generated earlier using the twitter feed

In [2]:
df = pd.read_csv('pacific.csv')
# Convert date column as datetime.
df['Date'] = pd.to_datetime(df['Date'] , format= '%Y%m%d')

### Status can be the following types:
1. TD – Tropical cyclone of tropical depression intensity (< 34 knots)
2. TS – Tropical cyclone of tropical storm intensity (34-63 knots)
3. HU – Tropical cyclone of hurricane intensity (> 64 knots)
4. EX – Extratropical cyclone (of any intensity)
5. SD – Subtropical cyclone of subtropical depression intensity (< 34 knots)
6. SS – Subtropical cyclone of subtropical storm intensity (> 34 knots)
7. LO – A low that is neither a tropical cyclone, a subtropical cyclone, nor an extratropical cyclone (of any intensity)
8. WV – Tropical Wave (of any intensity)
9. DB – Disturbance (of any intensity)

## PreProcessing data


### We are going to be doing the following preprocessing of the data:
1. Creating the column Latitude_Hemisphere. 
2. Convert the latitude and longitude Column to numeric type.
3. Show the count of missing values and fill them with mean.
4. Restructure the dataframe for visibility and remove columns ID and Event.        
5. Change all time to format HHMM.
6. Convert the column into Datetime.
7. Convert the status column to categorical.

In [3]:

# Create columns Latitude Hemisphere and Longitude Hemisphere with code 0 = N , 1 = S & 0 = E , 1 = W.
def hemisphere(coord):
        hem = re.findall(r'[NSWE]' , coord)[0]
        if hem == 'N' or hem == 'E':
            return 0
        else:
            return 1

# Creating the column Latitude_Hemisphere.    
df['Latitude_Hemisphere'] = df['Latitude'].apply(hemisphere)
df['Longitude_Hemisphere'] = df['Longitude'].apply(hemisphere)
df['Latitude_Hemisphere'] = df['Latitude_Hemisphere'].astype('category')
df['Longitude_Hemisphere'] = df['Longitude_Hemisphere'].astype('category')

# Convert the latitude and longitude Column to numeric type.
df['Latitude'] =  df['Latitude'].apply(lambda x: re.match('[0-9]{1,3}.[0-9]{0,1}' , x)[0])
df['Longitude'] =   df['Longitude'].apply(lambda x: re.match('[0-9]{1,3}.[0-9]{0,1}' , x)[0])

# The missing values are given by -999. So , we need to fill them appropriately.

# Show the count of missing values and fill them with mean.
for column in df.columns:
    missing_cnt = df[column][df[column] == -999].count()
    print('Missing Values in column {col} = '.format(col = column) , missing_cnt )
    if missing_cnt!= 0:
#         print('in ' , column)
        mean = round(df[column][df[column] != -999 ].mean())
#         print("mean",mean)
        index = df.loc[df[column] == -999 , column].index
#         print("index" , index )
        df.loc[df[column] == -999 , column] = mean
#         print(df.loc[index , column])
        
# Restructure the dataframe for visibility and remove columns ID and Event.        
df =  df[['ID', 'Name', 'Date', 'Time', 'Event', 'Status', 'Latitude', 'Latitude_Hemisphere' , 
       'Longitude', 'Longitude_Hemisphere' ,'Maximum Wind', 'Minimum Pressure', 'Low Wind NE',
       'Low Wind SE', 'Low Wind SW', 'Low Wind NW', 'Moderate Wind NE',
       'Moderate Wind SE', 'Moderate Wind SW', 'Moderate Wind NW',
       'High Wind NE', 'High Wind SE', 'High Wind SW', 'High Wind NW']]

# Change all time to format HHMM.
df['Time'] = df['Time'].astype('object')
def hhmm(time):
    time = str(time)
    digits = re.findall(r'\d', time)
    t = ''
    if len(digits) == 1:
        t ='0{i}00'.format(i =time)
    elif len(digits) == 2:
        t = '{i}00'.format(i =time)
    elif len(digits) == 3:
        t = '0{i}'.format(i =time)
    else:
        t = time
    return t
# Apply the function.
df['Time'] = df['Time'].apply(hhmm)

# Convert the column into Datetime.
df['Time'] = pd.to_datetime(df['Time'] , format='%H%M').dt.time


# Convert the status column to categorical.
df['Status'] = df['Status'].astype('category')

data = df.drop(columns = ['ID' , 'Event'])

Missing Values in column ID =  0
Missing Values in column Name =  0
Missing Values in column Date =  0
Missing Values in column Time =  0
Missing Values in column Event =  0
Missing Values in column Status =  0
Missing Values in column Latitude =  0
Missing Values in column Longitude =  0
Missing Values in column Maximum Wind =  0
Missing Values in column Minimum Pressure =  12804
Missing Values in column Low Wind NE =  19750
Missing Values in column Low Wind SE =  19750
Missing Values in column Low Wind SW =  19750
Missing Values in column Low Wind NW =  19750
Missing Values in column Moderate Wind NE =  19750
Missing Values in column Moderate Wind SE =  19750
Missing Values in column Moderate Wind SW =  19750
Missing Values in column Moderate Wind NW =  19750
Missing Values in column High Wind NE =  19750
Missing Values in column High Wind SE =  19750
Missing Values in column High Wind SW =  19750
Missing Values in column High Wind NW =  19750
Missing Values in column Latitude_Hemisp

## Adding new columns month and year

In [9]:
data['Month'] = data['Date'].apply(lambda x: x.month)
data['Year'] = data['Date'].apply(lambda x: x.year)
mnt = ['Jan' , 'Feb' , 'Mar' , 'Apr' , 'May' , 'June' , 'July' , 'Aug' , 'Sep','Oct' , 'Nov' , 'Dec']

## Display the data.


In [10]:
data.tail(10)

,Name,Date,Time,Status,Latitude,Latitude_Hemisphere,Longitude,Longitude_Hemisphere,Maximum Wind,Minimum Pressure,...,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW,Month,Year
26127,SANDRA,2015-11-27,06:00:00,HU,18.5,0,109.3,1,95,963,...,60,40,30,30,30,25,15,20,11,2015
26128,SANDRA,2015-11-27,12:00:00,HU,19.3,0,108.8,1,85,970,...,60,40,30,30,30,25,15,20,11,2015
26129,SANDRA,2015-11-27,18:00:00,HU,20.0,0,108.2,1,75,978,...,60,40,20,30,35,25,10,20,11,2015
26130,SANDRA,2015-11-28,00:00:00,TS,20.7,0,108.8,1,60,987,...,50,40,0,30,0,0,0,0,11,2015
26131,SANDRA,2015-11-28,06:00:00,LO,21.2,0,109.1,1,40,994,...,0,0,0,0,0,0,0,0,11,2015
26132,SANDRA,2015-11-28,12:00:00,LO,21.7,0,109.0,1,35,1002,...,0,0,0,0,0,0,0,0,11,2015
26133,SANDRA,2015-11-28,18:00:00,LO,22.4,0,108.7,1,30,1007,...,0,0,0,0,0,0,0,0,11,2015
26134,SANDRA,2015-11-29,00:00:00,LO,23.1,0,108.3,1,30,1008,...,0,0,0,0,0,0,0,0,11,2015
26135,SANDRA,2015-11-29,06:00:00,LO,23.5,0,107.9,1,25,1009,...,0,0,0,0,0,0,0,0,11,2015
26136,SANDRA,2015-11-29,12:00:00,LO,24.2,0,107.7,1,20,1010,...,0,0,0,0,0,0,0,0,11,2015


 # Classification model. 

## 1. Decision Tree.

1. Set the dependent and independent variables.
2. Perform the Kfold validation.
3. Train the Model using the training data
4. Calculate the accuracy of the model

In [11]:
# Import Decision Tree Classifier.
from sklearn.tree import DecisionTreeClassifier

# Import train-test split.
from sklearn.model_selection import train_test_split

# Import accuracy Score.
from sklearn.metrics import accuracy_score

#Import Recall Score.
from sklearn.metrics import recall_score 

#Import Precision Score.
from sklearn.metrics import precision_score 

# Form the model.
dt = DecisionTreeClassifier(min_samples_leaf=50 , criterion='entropy')


# Set the dependent and independent variables.
x_train = data[['Latitude', 'Latitude_Hemisphere',
       'Longitude', 'Longitude_Hemisphere', 'Maximum Wind', 'Minimum Pressure',
       'Low Wind NE', 'Low Wind SE', 'Low Wind SW', 'Low Wind NW',
       'Moderate Wind NE', 'Moderate Wind SE', 'Moderate Wind SW',
       'Moderate Wind NW', 'High Wind NE', 'High Wind SE', 'High Wind SW',
       'High Wind NW' , 'Month' , 'Year']]
y_train = data['Status']


# Perform the Kfold validation.

# Import the KFold library.
from sklearn.model_selection import KFold
kf = KFold(n_splits=10 , shuffle= True , random_state=42 )

dt_scores = []
dt_recall_scores = []
dt_precision_scores = []
for tr , ts in kf.split(x_train):
    xtr = x_train.loc[tr]
    ytr = y_train.loc[tr]
    xts = x_train.loc[ts]
    yts = y_train.loc[ts]
    dt.fit(xtr , ytr)
    y_pred = dt.predict(xts) 
    dt_scores.append(accuracy_score(yts, y_pred)) 
    dt_recall_scores.append(recall_score(yts , y_pred , average = 'weighted'))
    dt_precision_scores.append(precision_score(yts , y_pred , average = 'weighted'))
    
dt_scr = {'accuracy' : np.mean(dt_scores) , 'recall': np.mean(dt_recall_scores) , 'precision' :  np.mean(dt_precision_scores) }
print('Accuracy score for Decision Tree is :' , dt_scr['accuracy'])
print('Recall score for Decision Tree is :' , dt_scr['recall'])
print('Precision score for Decision Tree is :' , dt_scr['precision'])



Accuracy score for Decision Tree is : 0.9651068271145011
Recall score for Decision Tree is : 0.9651068271145011
Precision score for Decision Tree is : 0.9639969870825983


## 2. Random Forest.

1. Import Random Forest
2. Determine the important features.
3. Creating a decision tree for top ten most important features.
4. Set the dependent and independent variables.
5. Perform the Kfold validation.
6. Train the model
7. 

In [12]:
# Import Random Forest
from sklearn.ensemble import RandomForestClassifier

# Determine the important features.
rf = RandomForestClassifier(oob_score=True , n_estimators=1000)
rf.fit(x_train , y_train)
features = pd.Series(rf.feature_importances_ , index= x_train.columns).sort_values(ascending=False)
features

Maximum Wind            0.636030
Minimum Pressure        0.110219
Latitude                0.039106
Year                    0.038508
Longitude               0.037116
Low Wind NE             0.023915
Low Wind NW             0.018065
Low Wind SE             0.016749
High Wind NE            0.014496
Month                   0.011513
Low Wind SW             0.011082
High Wind SE            0.009138
High Wind NW            0.008662
Moderate Wind NE        0.005574
High Wind SW            0.005392
Moderate Wind SW        0.004190
Moderate Wind SE        0.004157
Moderate Wind NW        0.003878
Longitude_Hemisphere    0.002211
Latitude_Hemisphere     0.000000
dtype: float64

### Creating a decision tree for top ten most important features.

In [13]:
# Top ten most important features.
features.index[:5]

Index(['Maximum Wind', 'Minimum Pressure', 'Latitude', 'Year', 'Longitude'], dtype='object')

In [ ]:
# Set the dependent and independent variables.
x_trainf = data[features.index[:5]]
y_train = data['Status']

# Perform the Kfold validation.

# Import the KFold library.
from sklearn.model_selection import KFold
kf = KFold(n_splits=10 , shuffle= True , random_state=42 )

dt_scores = []
dt_recall_scores = []
dt_precision_scores = []
for tr , ts in kf.split(x_trainf):
    xtr = x_trainf.loc[tr]
    ytr = y_train.loc[tr]
    xts = x_trainf.loc[ts]
    yts = y_train.loc[ts]
    dt.fit(xtr , ytr)
    y_pred = dt.predict(xts) 
    dt_scores.append(accuracy_score(yts, y_pred)) 
    dt_recall_scores.append(recall_score(yts , y_pred , average = 'weighted'))
    dt_precision_scores.append(precision_score(yts , y_pred , average = 'weighted'))

dt_scr5 = {'accuracy' : np.mean(dt_scores) , 'recall': np.mean(dt_recall_scores) , 'precision' :  np.mean(dt_precision_scores) }
print('Accuracy score for Decision Tree is :' , dt_scr['accuracy'])
print('Recall score for Decision Tree is :' , dt_scr['recall'])
print('Precision score for Decision Tree is :' , dt_scr['precision'])

#### As we can see the Top five features('Maximum Wind', 'Minimum Pressure', 'Latitude', 'Year', 'Longitude') give the same accuracy as when we get choosing all the features.

## 2 . Random Forest

In [ ]:
# Here instead of cross validation we will be using oob score as a measure of accuracy.
# I will hyper tuning the parameter: No of Trees.

trees  = [10, 20 , 50, 100,200,500,1000,1200]
maxn_five = {}
maxn = {}
for i in trees:
    rf = RandomForestClassifier(n_estimators=i , oob_score=True)
    rf.fit(x_trainf , y_train)
    print('Obb Score for {x} trees: and taking top five features '.format(x = i) , rf.oob_score_)
    maxn_five[i] = rf.oob_score_
    rf.fit(x_trainf , y_train)
    print('Obb Score for {x} trees: and taking all the features '.format(x = i) , rf.oob_score_)
    maxn[i] = rf.oob_score_

In [ ]:
# Split the data into training and testing.
x_trains , x_tests , y_trains, y_tests  = train_test_split(x_trainf, y_train, test_size=0.33, random_state=42)
# Set n to the feature of maximum oob score.
n = 0
for i in maxn_five:
    if max(maxn_five.values()) == maxn_five[i]:
        n= i
# Set n_estimators to n.
rf = RandomForestClassifier(oob_score=True , n_estimators=n)
rf.fit(x_trains , y_trains)
y_pred_rf = rf.predict(x_tests[features.index[:5]])
scores_rf = {'accuracy': accuracy_score(y_tests , y_pred_rf) ,'recall' : recall_score(y_tests , y_pred_rf , average='weighted') ,'precision' : precision_score(y_tests , y_pred_rf , average='weighted') }
print('Scores for Random Forest with n = ' , n , ' and using features ',  features.index[:5] , ' are : ')
print('Accuracy: ' , scores_rf['accuracy'])
print('Recall: ' , scores_rf['recall'])
print('Precision: ' , scores_rf['precision'])


#### We can see that the overall score with top five features is significantly greater than the overall score with all the features. Hence , we can see that feature selection is very important for Random Forest.